In [1]:
## Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import base64

# Configure OS routines
import os

# Configure the plotting routines
import pandas as pd

# Import the CRUD module for MongoDB
from AnimalShelter import AnimalShelter  

# Connect to the database using credentials
username = "aacuser"
password = "SNHU1234"
db = AnimalShelter(username, password)

# Load initial data from MongoDB
df = pd.DataFrame.from_records(db.read({}))

# Remove MongoDB's default '_id' field
if "_id" in df.columns:
    df.drop(columns=["_id"], inplace=True)

# Define breed-to-rescue mapping
breed_roles = {
    "Labrador Retriever": "Water Rescue",
    "Newfoundland": "Water Rescue",
    "German Shepherd": "Mountain or Wilderness Rescue",
    "Border Collie": "Mountain or Wilderness Rescue",
    "Bloodhound": "Disaster or Individual Tracking",
    "Belgian Malinois": "Disaster or Individual Tracking"
}

# Initialize Dash app
app = JupyterDash(__name__)

# Encode the Grazioso Salvare logo
image_filename = 'grazioso.png'  
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

# Define the app layout
app.layout = html.Div([
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Img(src=f"data:image/png;base64,{encoded_image}", style={'width': '200px'}),
    html.P("Dashboard by: Lilly"),  # Unique identifier
    html.Hr(),

    # Interactive Filtering Options
    html.Label("Filter by Rescue Type"),
    dcc.Dropdown(
        id='filter-type',
        options=[
            {'label': 'All', 'value': 'all'},
            {'label': 'Water Rescue', 'value': 'Water Rescue'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
            {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'}
        ],
        value='all',  # Default to showing all data
        multi=False
    ),

    html.Br(),

    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        style_table={'overflowX': 'auto'},
        row_selectable='single',
        selected_rows=[0]
    ),

    html.Br(),
    html.Hr(),

    # Dashboard layout: Chart and Map side by side
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 html.Div(id='graph-id', className='col s12 m6'),
                 html.Div(id='map-id', className='col s12 m6')
             ])
])

#############################################
# Interaction Between Components / Controller
#############################################


# Callback to update the table data based on filtering selection
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    # Define filtering rules for each rescue type
    rescue_filters = {
        "Water Rescue": {
            "breed": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"],
            "sex_upon_outcome": {"$regex": "Intact Female"},
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        },
        "Mountain or Wilderness Rescue": {
            "breed": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"],
            "sex_upon_outcome": {"$regex": "Intact Male"},
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        },
        "Disaster or Individual Tracking": {
            "breed": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"],
            "sex_upon_outcome": {"$regex": "Intact Male"},
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }
    }

    # If 'All' is selected, return all dogs under 2 years old
    if filter_type == 'all':
        query = {"age_upon_outcome_in_weeks": {"$lt": 104}} #To stop the virtual access from crashing
    else:
        # Apply breed, sex, and age filters based on selected type
        query = {
            "breed": {"$in": rescue_filters[filter_type]["breed"]},
            "sex_upon_outcome": rescue_filters[filter_type]["sex_upon_outcome"],
            "age_upon_outcome_in_weeks": rescue_filters[filter_type]["age_upon_outcome_in_weeks"]
        }

    df_filtered = pd.DataFrame.from_records(db.read(query))

    if "_id" in df_filtered.columns:
        df_filtered.drop(columns=["_id"], inplace=True)

    return df_filtered.to_dict('records')



# Callback to update breed distribution bar chart
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    if not viewData:
        return "No data available"

    dff = pd.DataFrame.from_dict(viewData)

    # Ensure 'breed' column exists
    if 'breed' not in dff.columns or dff['breed'].isnull().all():
        return "No breed data available"

    # Count occurrences of each breed
    breed_counts = dff['breed'].value_counts().reset_index()
    breed_counts.columns = ['Breed', 'Count']

    # Set a threshold to group small breeds into "Other"
    threshold = 1  # Any breed with <1% will be grouped into "Other"
    total_count = breed_counts['Count'].sum()

    # Identify small breeds
    breed_counts['Percentage'] = (breed_counts['Count'] / total_count) * 100
    small_breeds = breed_counts[breed_counts['Percentage'] < threshold]

    # Sum counts for "Other" category
    other_count = small_breeds['Count'].sum()

    # Keep only the top 10 breeds, then group remaining ones into "Other"
    top_n = 10
    breed_counts = breed_counts.sort_values('Count', ascending=False)

    if len(breed_counts) > top_n:
        breed_counts = breed_counts.iloc[:top_n]
        breed_counts = breed_counts.append({'Breed': 'Other', 'Count': other_count}, ignore_index=True)

    # Create the pie chart
    fig = px.pie(
        breed_counts, 
        names='Breed', 
        values='Count', 
        title="Breed Distribution (Top 10 + Other)",
        color_discrete_sequence=px.colors.qualitative.Set3
    )

    # Adjust text position to avoid clutter
    fig.update_traces(textposition='inside', textinfo='percent+label')

    return dcc.Graph(figure=fig)


# Callback to update the geolocation map
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_map(viewData):
    if not viewData:
        return "No data available"

    dff = pd.DataFrame.from_dict(viewData)

    # Generate markers for each dog
    markers = [
        dl.Marker(
            position=[row.get('latitude', 30.75), row.get('longitude', -97.48)],
            children=[
                dl.Tooltip(row.get('breed', 'Unknown')),
                dl.Popup([html.H1(row.get('name', 'Unknown')), html.P(f"Age: {row.get('age_upon_outcome', 'Unknown')}")])
            ]
        )
        for _, row in dff.iterrows()
    ]

    return dl.Map(
        style={'width': '1000px', 'height': '500px'},
        center=[30.75, -97.48], zoom=10, 
        children=[dl.TileLayer(id="base-layer-id")] + markers
    )


# Run the Jupyter Dash app
app.run_server(debug=True, port=8051)



/home/lillianberry_snhu
MongoDB connection successful.
Dash app running on http://127.0.0.1:8051/


/tmp/ipykernel_137096/3852325871.py:180: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

